# Try to get all hydrogens around metal atoms about a conventional cell

In [97]:
from pymatgen.core.structure import Structure
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer
from collections import defaultdict
# pmg_struct = Structure.from_file("/home/mayuan/mycode/my_script/test/clathrate/CaH6.vasp")
# pmg_struct = Structure.from_file("/home/mayuan/mycode/my_script/test/clathrate/LaH10.vasp")
# pmg_struct = Structure.from_file("/home/mayuan/mycode/my_script/test/clathrate/LaBH8.vasp")
# pmg_struct = Structure.from_file("/home/mayuan/mycode/my_script/test/clathrate/Li2MgH16.vasp")
# pmg_struct = Structure.from_file("/home/mayuan/mycode/my_script/test/clathrate/ScH9.vasp")
pmg_struct = Structure.from_file("/home/mayuan/mycode/my_script/test/clathrate/CaYH12.vasp")
# for site in pmg_struct.sites:
#     print(site)
center_indices, points_indices, images, distances = pmg_struct.get_neighbor_list(
    r=2.2, sites=pmg_struct.sites, numerical_tol=1e-8
)
big_cage = defaultdict(int)
for center, points, imgs, dist in zip(center_indices, points_indices, images, distances):
    if str(pmg_struct.sites[center].specie) != "H" and 1.7 <= dist <= 2.2 :
        big_cage[str(pmg_struct.sites[center].specie)] += 1

small_cage = defaultdict(int)
for center, points, imgs, dist in zip(center_indices, points_indices, images, distances):
    if str(pmg_struct.sites[center].specie) != "H" and 1.3 <= dist <= 1.8 :
        small_cage[str(pmg_struct.sites[center].specie)] += 1

el_amt = pmg_struct.composition.get_el_amt_dict()

amounts = {}
specie_without_H = list(el_amt.keys())
specie_without_H.remove("H")
for key in specie_without_H:
    from_big_cage = big_cage.get(key, 0)
    from_small_cage = small_cage.get(key, 0)
    if from_big_cage > from_small_cage:
        amounts[key] = from_big_cage
    else:
        amounts[key] = from_small_cage
    
Hcage = defaultdict(int)
for key, value in amounts.items():
    if key in list(el_amt.keys()):
        around = value / el_amt[key]
        Hcage[key] = around

print(Hcage)

defaultdict(<class 'int'>, {'Ca': 24.0, 'Y': 24.0})


In [87]:
el_amt = pmg_struct.composition.get_el_amt_dict()
print(el_amt)
specie_without_H = list(el_amt.keys())
specie_without_H.remove("H")
specie_without_H

defaultdict(<class 'float'>, {'Li': 16.0, 'Mg': 8.0, 'H': 128.0})


['Li', 'Mg']

# Count the amount of a specie in a cell

In [88]:
from pymatgen.core.structure import Structure
pmg_struct = Structure.from_file("/home/mayuan/mycode/my_script/test/clathrate/Li2MgH16.vasp")
pmg_struct.composition.to_data_dict
# {
#   'reduced_cell_composition': Comp: Li2 Mg1 H16,
#   'unit_cell_composition': defaultdict(float,
#              {'Li': 16.0, 'Mg': 8.0, 'H': 128.0}),
#   'reduced_cell_formula': 'Li2MgH16',
#   'elements': ['Li', 'Mg', 'H'],
#   'nelements': 3
# }
pmg_struct.composition.num_atoms            # get amounts of all atoms
pmg_struct.composition.alphabetical_formula # 'H128 Li16 Mg8'
pmg_struct.composition.anonymized_formula   # 'AB2C16'
pmg_struct.composition.iupac_formula        # 'Li16 Mg8 H128'
pmg_struct.composition.chemical_system      # 'H-Li-Mg'
pmg_struct.composition.elements             # [Element Li, Element Mg, Element H]
pmg_struct.composition.special_formulas     #{
                                            #     'LiO': 'Li2O2',
                                            #     'NaO': 'Na2O2',
                                            #     'KO': 'K2O2',
                                            #     'HO': 'H2O2',
                                            #     'CsO': 'Cs2O2',
                                            #     'RbO': 'Rb2O2',
                                            #     'O': 'O2',
                                            #     'N': 'N2',
                                            #     'F': 'F2',
                                            #     'Cl': 'Cl2',
                                            #     'H': 'H2'
                                            # }
pmg_struct.composition.get_el_amt_dict()                    # defaultdict(float, {'Li': 16.0, 'Mg': 8.0, 'H': 128.0})
pmg_struct.composition.get_integer_formula_and_factor()     # ('Li2MgH16', 8.0)
pmg_struct.composition.get_reduced_composition_and_factor() # (Comp: Li2 Mg1 H16, 8)


(Comp: Li2 Mg1 H16, 8)

# **一个小工具** 输入一个结构，输出其约化化学式，空间群对称性，文件名

In [1]:
import sys
from pathlib import Path

from pymatgen.core.structure import Structure
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer

path = Path(sys.argv[1])
file_name = path.name
struct = Structure.from_file(path)
name = struct.composition.get_integer_formula_and_factor()[0]
num = SpacegroupAnalyzer(struct).get_space_group_number()
spg_symbol = SpacegroupAnalyzer(struct).get_space_group_symbol()
print(f"{num}-{spg_symbol}-{name}")
print(f"{file_name}")

FileNotFoundError: [Errno 2] No such file or directory: '-f'